#### Step 01 - Create EXTERNAL LOCATION
Em termos práticos: “Essa pasta no meu Data Lake existe, está sob controle, e aqui estão as credenciais e permissões para acessá-la.”

Ele serve para conectar um caminho físico no ADLS (ou outro storage) a um objeto lógico no Unity Catalog, com governança e segurança centralizadas.

In [0]:
%sql
-- Raw 
CREATE EXTERNAL LOCATION IF NOT EXISTS finance_raw
  URL 'abfss://finance@personalfinancedl.dfs.core.windows.net/raw'
  WITH (STORAGE CREDENTIAL vale_sc)
  COMMENT 'Dados brutos de entrada (raw layer)';

-- Landing (somente leitura para ETL)
CREATE EXTERNAL LOCATION IF NOT EXISTS finance_landing
  URL 'abfss://finance@personalfinancedl.dfs.core.windows.net/landing'
  WITH (STORAGE CREDENTIAL vale_sc)
  COMMENT 'Dados brutos de entrada (landing layer)';

-- Bronze
CREATE EXTERNAL LOCATION IF NOT EXISTS finance_bronze
  URL 'abfss://finance@personalfinancedl.dfs.core.windows.net/bronze'
  WITH (STORAGE CREDENTIAL vale_sc)
  COMMENT 'Camada bronze (dados raw normalizados)';

-- Silver
CREATE EXTERNAL LOCATION IF NOT EXISTS finance_silver
  URL 'abfss://finance@personalfinancedl.dfs.core.windows.net/silver'
  WITH (STORAGE CREDENTIAL vale_sc)
  COMMENT 'Camada silver (dados refinados)';

-- Gold
CREATE EXTERNAL LOCATION IF NOT EXISTS finance_gold
  URL 'abfss://finance@personalfinancedl.dfs.core.windows.net/gold'
  WITH (STORAGE CREDENTIAL vale_sc)
  COMMENT 'Camada gold (dados prontos para consumo)';

-- Unity Catalog
CREATE EXTERNAL LOCATION IF NOT EXISTS finance_uc_managed
  URL 'abfss://finance@personalfinancedl.dfs.core.windows.net/_uc_managed'
  WITH (STORAGE CREDENTIAL vale_sc)
  COMMENT 'Área dedicada para Managed Tables do catálogo personalfinance';

#### Step 02 - Create CATALOG
O CREATE CATALOG registra no Unity Catalog um “container” de dados com um endereço físico no seu Data Lake, para que você possa criar schemas e tabelas dentro dele com segurança, organização e controle de acesso centralizado. Cada catálogo tem um Managed Location opcional, que define onde ficam fisicamente os dados das tabelas managed criadas nele.

CATALOG  →  SCHEMA  →  TABELA / VIEW / VOLUME

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS personalfinance
    MANAGED LOCATION 'abfss://finance@personalfinancedl.dfs.core.windows.net/_uc_managed'
    COMMENT 'Catálogo central do Lakehouse Personal Finance';

#### Step 03 - Create SCHEMAS
O CREATE SCHEMA cria um espaço organizado dentro do catálogo, com um endereço físico no Data Lake, para guardar objetos de dados de forma isolada e com controle de acesso específico. Com MANAGED LOCATION, você aponta para a pasta no Data Lake onde o Unity Catalog vai armazenar os dados managed desse schema.

**Managed Location do catálogo:** É o endereço padrão para qualquer schema que não tenha um local próprio. Se você criar um schema sem MANAGED LOCATION, ele herda o caminho do catálogo.

**Managed Location do schema:** É um endereço dedicado para os dados managed daquele schema. Ele sobrescreve o caminho padrão do catálogo e garante que as tabelas daquele schema sejam gravadas em uma pasta específica no Data Lake.

Assim, quando você usar df.saveAsTable('valeparaopeba.silver.tabela_x'), os arquivos serão salvos dentro da pasta silver, como você espera.

In [0]:
%sql
-- Camada Bronze: dados brutos normalizados, primeira etapa do pipeline ETL
CREATE SCHEMA IF NOT EXISTS personalfinance.bronze
  MANAGED LOCATION 'abfss://finance@personalfinancedl.dfs.core.windows.net/bronze'
  COMMENT 'Camada bronze: dados brutos normalizados, utilizados como base para refinamento';

-- Camada Silver: dados tratados e enriquecidos, prontos para análises intermediárias
CREATE SCHEMA IF NOT EXISTS personalfinance.silver
  MANAGED LOCATION 'abfss://finance@personalfinancedl.dfs.core.windows.net/silver'
  COMMENT 'Camada silver: dados tratados e integrados, com qualidade e consistência para análises';

-- Camada Gold: dados refinados e agregados, prontos para consumo por BI e stakeholders
CREATE SCHEMA IF NOT EXISTS personalfinance.gold
  MANAGED LOCATION 'abfss://finance@personalfinancedl.dfs.core.windows.net/gold'
  COMMENT 'Camada gold: dados finais e otimizados para consumo por dashboards, relatórios e modelos';

-- Camada Landing: área de ingestão bruta, acessada via External Volumes
CREATE SCHEMA IF NOT EXISTS personalfinance.landing
  COMMENT 'Camada landing: área de ingestão de dados brutos, acessada via External Volumes para leitura';

In [0]:
%sql
DESCRIBE SCHEMA personalfinance.landing;

#### Step 04 - Create VOLUMES
Um External Volume é um recurso do Unity Catalog que permite acessar arquivos diretamente no Data Lake, sem precisar criar uma tabela. Ele funciona como uma “ponte” entre o catálogo e os arquivos brutos (.parquet, .csv, .json, etc.) que estão armazenados em diretórios externos.

LOCATION > não gerencia os arquivos (não é MANAGED LOCATION), ele apenas referencia um diretório externo no seu Data Lake.

In [0]:
%sql
CREATE EXTERNAL VOLUME IF NOT EXISTS personalfinance.landing.vol_landing
  LOCATION 'abfss://finance@personalfinancedl.dfs.core.windows.net/landing'
  COMMENT 'Volume externo para leitura de arquivos brutos da camada landing';